1. importing the paraphrased prompts

In [8]:
import pandas as pd
data = pd.DataFrame(pd.read_excel("/Users/paul/Desktop/Eulith_AI_Agent/annotation_sets/WETH_UNI_SWAP.xlsx"))
#The rows describe actions as the following:
#0 to 99 : Swap WETH to UNI
#100 to 208 : Swap UNI to WETH
#209 to 317 : Swap USDC to WETH
#318 to 421 : Swap WETH to USDC

,"""Set up a WETH and UNI swap"""
0,"""Initiate a swap between WETH and UNI"""
1,"""Perform a WETH to UNI swap"""
2,"""Exchange WETH for UNI"""
3,"""Swap WETH with UNI"""
4,"""Conduct a WETH-UNI swap"""
...,...
417,"""Start a WETH and USDC swap without bells and ..."
418,"""Prepare a WETH-USDC exchange without extras"""
419,"""Organize a bare-bones WETH and USDC swap"""
420,"""Implement a WETH-USDC trade with no frills"""


2. cretaing general completions

In [12]:
pd.DataFrame(
    {'Operation': ["Swap_UNI_to_WETH",
                   "Swap_WETH_to_UNI",
                   "Swap_USDC_to_WETH",
                   "Swap_WETH_to_USDC"
                   ],
     'completion': ["from eulith_web3.erc20\n import TokenSymbol\nfrom eulith_web3.eulith_web3 import *\nfrom eulith_web3.signing import construct_signing_middleware, LocalSigner\n\nif __name__ == '__main__':\n    wallet = LocalSigner(\"...\") # YOUR PRIVATE KEY GOES HERE\n    ew3 = EulithWeb3(eulith_url=\"...\", #URL ETHEREUM VIRTUAL MACHINE GOES HERE \n                     eulith_refresh_token=\"...\", # YOUR REFRESH TOKEN GOES HERE\n                     signing_middle_ware=construct_signing_middleware(wallet)) \n   \n   \n    # these are python bindings around the whole ERC20 contract\n    weth = ew3.eulith_get_erc_token(TokenSymbol.WETH)\n    uni = ew3.eulith_get_erc_token(TokenSymbol.UNI)\n    \n    _sell_amount=0.001 # just an arbitrary small number for example\n    \n    # set the swap parameters\n    swap_params = EulithSwapRequest(\n        sell_token=weth,\n        buy_token=uni,\n        sell_amount=_sell_amount, # quantity; no weird decimals, 0.001 means 0.001 WETH\n    )\n    \n    # get a swap quote\n    # txs is an array of transactions that make up the swap\n    quote, txs = ew3.eulith_swap_quote(swap_params)\n    \n    # convert eth to weth to prepare for the swap\n    eth_to_weth_tx = weth.deposit_eth(_sell_amount)\n    eth_to_weth_tx['from'] = wallet.address\n    ew3.eth.send_transaction(eth_to_weth_tx)\n    \n    # if you decide you like the quote, send the swap transactions\n    # this function is used to take an array of transactions and batch them atomically\n    # does not get sent to the blockchain\n    ew3.eulith_send_multi_transaction(txs)",
                    "from eulith_web3.erc20\n import TokenSymbol\nfrom eulith_web3.eulith_web3 import *\nfrom eulith_web3.signing import construct_signing_middleware, LocalSigner\n\nif __name__ == '__main__':\n    wallet = LocalSigner(\"...\") # YOUR PRIVATE KEY GOES HERE\n    ew3 = EulithWeb3(eulith_url=\"...\", #URL ETHEREUM VIRTUAL MACHINE GOES HERE \n                     eulith_refresh_token=\"...\", # YOUR REFRESH TOKEN GOES HERE\n                     signing_middle_ware=construct_signing_middleware(wallet)) \n   \n   \n    # these are python bindings around the whole ERC20 contract\n    uni = ew3.eulith_get_erc_token(TokenSymbol.UNI)\n    weth = ew3.eulith_get_erc_token(TokenSymbol.WETH)\n    \n    _sell_amount=0.001 # just an arbitrary small number for example\n    \n    # set the swap parameters\n    swap_params = EulithSwapRequest(\n        sell_token=uni,\n        buy_token=weth,\n        sell_amount=_sell_amount, # quantity; no weird decimals, 0.001 means 0.001 WETH\n    )\n    \n    # get a swap quote\n    # txs is an array of transactions that make up the swap\n    quote, txs = ew3.eulith_swap_quote(swap_params)\n    \n    # convert eth to weth to prepare for the swap\n    eth_to_weth_tx = weth.deposit_eth(_sell_amount)\n    eth_to_weth_tx['from'] = wallet.address\n    ew3.eth.send_transaction(eth_to_weth_tx)\n    \n    # if you decide you like the quote, send the swap transactions\n    # this function is used to take an array of transactions and batch them atomically\n    # does not get sent to the blockchain\n    ew3.eulith_send_multi_transaction(txs)",
                    "from eulith_web3.erc20\n import TokenSymbol\nfrom eulith_web3.eulith_web3 import *\nfrom eulith_web3.signing import construct_signing_middleware, LocalSigner\n\nif __name__ == '__main__':\n    wallet = LocalSigner(\"...\") # YOUR PRIVATE KEY GOES HERE\n    ew3 = EulithWeb3(eulith_url=\"...\", #URL ETHEREUM VIRTUAL MACHINE GOES HERE \n                     eulith_refresh_token=\"...\", # YOUR REFRESH TOKEN GOES HERE\n                     signing_middle_ware=construct_signing_middleware(wallet)) \n   \n   \n    # these are python bindings around the whole ERC20 contract\n    usdc = ew3.eulith_get_erc_token(TokenSymbol.USDC)\n    weth = ew3.eulith_get_erc_token(TokenSymbol.WETH)\n    \n    _sell_amount=0.001 # just an arbitrary small number for example\n    \n    # set the swap parameters\n    swap_params = EulithSwapRequest(\n        sell_token=usdc,\n        buy_token=weth,\n        sell_amount=_sell_amount, # quantity; no weird decimals, 0.001 means 0.001 WETH\n    )\n    \n    # get a swap quote\n    # txs is an array of transactions that make up the swap\n    quote, txs = ew3.eulith_swap_quote(swap_params)\n    \n    # convert eth to weth to prepare for the swap\n    eth_to_weth_tx = weth.deposit_eth(_sell_amount)\n    eth_to_weth_tx['from'] = wallet.address\n    ew3.eth.send_transaction(eth_to_weth_tx)\n    \n    # if you decide you like the quote, send the swap transactions\n    # this function is used to take an array of transactions and batch them atomically\n    # does not get sent to the blockchain\n    ew3.eulith_send_multi_transaction(txs)",
                    "from eulith_web3.erc20\n import TokenSymbol\nfrom eulith_web3.eulith_web3 import *\nfrom eulith_web3.signing import construct_signing_middleware, LocalSigner\n\nif __name__ == '__main__':\n    wallet = LocalSigner(\"...\") # YOUR PRIVATE KEY GOES HERE\n    ew3 = EulithWeb3(eulith_url=\"...\", #URL ETHEREUM VIRTUAL MACHINE GOES HERE \n                     eulith_refresh_token=\"...\", # YOUR REFRESH TOKEN GOES HERE\n                     signing_middle_ware=construct_signing_middleware(wallet)) \n   \n   \n    # these are python bindings around the whole ERC20 contract\n    weth = ew3.eulith_get_erc_token(TokenSymbol.WETH)\n    usdc = ew3.eulith_get_erc_token(TokenSymbol.USDC)\n    \n    _sell_amount=0.001 # just an arbitrary small number for example\n    \n    # set the swap parameters\n    swap_params = EulithSwapRequest(\n        sell_token=weth,\n        buy_token=usdc,\n        sell_amount=_sell_amount, # quantity; no weird decimals, 0.001 means 0.001 WETH\n    )\n    \n    # get a swap quote\n    # txs is an array of transactions that make up the swap\n    quote, txs = ew3.eulith_swap_quote(swap_params)\n    \n    # convert eth to weth to prepare for the swap\n    eth_to_weth_tx = weth.deposit_eth(_sell_amount)\n    eth_to_weth_tx['from'] = wallet.address\n    ew3.eth.send_transaction(eth_to_weth_tx)\n    \n    # if you decide you like the quote, send the swap transactions\n    # this function is used to take an array of transactions and batch them atomically\n    # does not get sent to the blockchain\n    ew3.eulith_send_multi_transaction(txs)"
                    ]

     }
)


,Operation,completion
0,Swap_UNI_to_WETH,from eulith_web3.erc20\n import TokenSymbol\nf...
1,Swap_WETH_to_UNI,from eulith_web3.erc20\n import TokenSymbol\nf...
2,Swap_USDC_to_WETH,from eulith_web3.erc20\n import TokenSymbol\nf...
3,Swap_WETH_to_USDC,from eulith_web3.erc20\n import TokenSymbol\nf...
